python setup.py clean --all

python setup.py sdist bdist_wheel

twine upload --skip-existing dist/*

In [1]:
import sys
sys.path.append('..')
import ScraperFC as sfc
import traceback
import pandas as pd
import numpy as np

In [4]:
scraper = sfc.FBRef()
try:
#     link = 'https://fbref.com/en/matches/b3f1bb11/Sporting-KC-Minnesota-United-November-1-2020-Major-League-Soccer'
    link = 'https://fbref.com/en/matches/4d591949/New-England-Revolution-DC-United-November-1-2020-Major-League-Soccer'
    match = scraper.scrape_match(link)
except:
    traceback.print_exc()
scraper.close()

[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 104.0.5112
[WDM] - Get LATEST driver version for 104.0.5112
[WDM] - Driver [C:\Users\Owner\.wdm\drivers\chromedriver\win32\104.0.5112.79\chromedriver.exe] found in cache


In [5]:
match

,Link,Date,Matchweek,Home Team,Away Team,Home Team ID,Away Team ID,Home Formation,Away Formation,Home Goals,...,Away Ast,Home xG,Away xG,Home npxG,Away npxG,Home xA,Away xA,Home Player Stats,Away Player Stats,Shots
0,https://fbref.com/en/matches/4d591949/New-Engl...,2020-11-01,Regular Season,New England Revolution,D.C. United,3c079def,44117292,4-2-3-1,4-4-2,4,...,2,3.4,1.0,2.4,1.0,1.9,0.8,Team Sheet New England (4-2-3-1) New Eng...,Team Sheet D.C. United (4-4-2) D.C. Unit...,Both Unnamed: 0_level_0 Unnamed: 1_level...


In [4]:
# import requests
# from bs4 import BeautifulSoup

# response = requests.get(link)
# soup = BeautifulSoup(response.content, 'html.parser')


# int(scores[0].getText())

NameError: name 'scores' is not defined